In [7]:
import math
import random
from typing import List, Dict, Tuple, Optional
from dataclasses import dataclass

class MaxCutInstance:
    def __init__(self):
        self.edges: Dict[Tuple[int, int], float] = {}
        self.size: int = 0
    
    def get_size(self) -> int:
        return self.size
    
    def get_all_edges(self):
        return self.edges.items()

class MaxCutSolution:
    def __init__(self, instance: MaxCutInstance, heuristic=None):
        self.mi = instance
        self.N_ = instance.get_size()
        self.assignments_: List[int] = [-1] * self.N_
        self.diff_weights_: List[float] = [0.0] * self.N_
        self.weight_: float = 0.0
        self.heuristic = heuristic

    def get_assignments(self) -> List[int]:
        return self.assignments_

    def get_weight(self) -> float:
        return self.weight_

    def improves_over(self, other_weight: float) -> bool:
        return self.weight_ > other_weight

class Burer2002Solution(MaxCutSolution):
    def __init__(self, mi: MaxCutInstance, w1norm: float, theta: List[float], heuristic=None):
        super().__init__(mi, heuristic)
        self._construct_solution(w1norm, theta)

    @staticmethod
    def rank2cut(mi: MaxCutInstance, w1norm: float, theta: List[float], heuristic=None):
        return Burer2002Solution(mi, w1norm, theta, heuristic)

    def all_1_swap(self, tolerance: float):
        move_made = True
        while move_made:
            move_made = False
            for i in range(self.N_):
                if self.diff_weights_[i] > tolerance and self._improving_move(i):
                    self._update_cut_values(i)
                    move_made = True
                    break

    def all_2_swap(self, tolerance: float):
        move_made = True
        while move_made:
            move_made = False
            for (i, j), w_ij in self.mi.get_all_edges():
                benefit = (self.diff_weights_[i] + self.diff_weights_[j] - 
                         2.0 * self.assignments_[i] * self.assignments_[j] * w_ij)
                if benefit > tolerance and self._improving_move(benefit):
                    self._update_cut_values(i)
                    self._update_cut_values(j)
                    move_made = True
                    break

    def _load_new_theta(self, theta: List[float], cos_theta: List[float], 
                       sin_theta: List[float], dH: List[float]) -> float:
        for i in range(self.N_):
            cos_theta[i] = math.cos(theta[i])
            sin_theta[i] = math.sin(theta[i])
            dH[i] = 0.0

        objective = 0.0
        for (i, j), w_ij in self.mi.get_all_edges():
            scaled_cos_diff = w_ij * (
                cos_theta[i] * cos_theta[j] + sin_theta[i] * sin_theta[j]
            )
            objective += scaled_cos_diff
            dH[i] -= scaled_cos_diff
            dH[j] -= scaled_cos_diff
        return objective

    def _construct_solution(self, w1norm: float, theta: List[float]):
        # Parameters
        F_TOLERANCE = 1e-4
        G_TOLERANCE = 1e-4
        MAX_BACK = 10
        TAU = 0.5
        GAMMA = 0.01
        ALPHA_INIT = 1.0
        MAX_ALPHA = 4.0
        MAX_OPT_ITER = 200

        # Setup variables
        bt_alpha = ALPHA_INIT
        dH = [0.0] * self.N_
        cos_theta = [0.0] * self.N_
        sin_theta = [0.0] * self.N_
        
        f = self._load_new_theta(theta, cos_theta, sin_theta, dH)

        for opt_iter in range(MAX_OPT_ITER):
            # Compute current gradient
            g = [0.0] * self.N_
            for (i, j), w_ij in self.mi.get_all_edges():
                scaled_sin_diff = w_ij * (
                    sin_theta[j] * cos_theta[i] - cos_theta[j] * sin_theta[i]
                )
                g[i] += scaled_sin_diff
                g[j] -= scaled_sin_diff

            # Check gradient stopping condition
            norm_gradient = sum(x * x for x in g)
            if norm_gradient / w1norm < G_TOLERANCE:
                break

            # Determine descent direction
            desc = [0.0] * self.N_
            g_times_desc = 0.0
            divisor = max(1.0, max(dH))
            
            for ct in range(self.N_):
                desc[ct] = -g[ct] / divisor
                g_times_desc += desc[ct] * g[ct]

            # Line search
            new_theta = [0.0] * self.N_
            recent_f = -1.0
            
            for numback in range(1, MAX_BACK + 1):
                for ct in range(self.N_):
                    new_theta[ct] = theta[ct] + bt_alpha * desc[ct]

                recent_f = self._load_new_theta(new_theta, cos_theta, sin_theta, dH)
                if recent_f <= f + GAMMA * bt_alpha * g_times_desc:
                    break

                bt_alpha *= TAU

            f_prev = f
            f = recent_f
            theta[:] = new_theta

            # Check objective value stopping condition
            rel_change = abs(f - f_prev) / (1.0 + abs(f_prev))
            if rel_change < F_TOLERANCE:
                break

            # Update step size
            if numback <= 2:
                if MAX_ALPHA < 2.0 * bt_alpha:
                    bt_alpha = MAX_ALPHA
                else:
                    bt_alpha *= 2.0

        # Process angles and create cut
        angles = []
        for ct in range(self.N_):
            theta[ct] -= 2 * math.pi * math.floor(theta[ct] / (2 * math.pi))
            angles.append((theta[ct], ct))
        angles.append((2 * math.pi, self.N_))
        angles.sort()

        # Initialize assignments
        self.assignments_ = [-1] * self.N_
        first_it = 0
        second_it = 0

        while angles[second_it][0] <= math.pi:
            self.assignments_[angles[second_it][1]] = 1
            second_it += 1

        self._populate_from_assignments()
        curr_weight = self.weight_
        curr_assignments = self.assignments_.copy()
        curr_diff_weights = self.diff_weights_.copy()

        # Find optimal cut
        while True:
            if angles[first_it][0] <= angles[second_it][0] - math.pi:
                update_index = angles[first_it][1]
                first_it += 1
            else:
                update_index = angles[second_it][1]
                second_it += 1

            if update_index == self.N_:
                break

            self._update_cut_values(update_index, curr_assignments, 
                                  curr_diff_weights, curr_weight)
            
            if self._improves_over(curr_weight, self.weight_):
                self.weight_ = curr_weight
                self.assignments_ = curr_assignments.copy()
                self.diff_weights_ = curr_diff_weights.copy()

    def _populate_from_assignments(self):
        self.weight_ = 0.0
        self.diff_weights_ = [0.0] * self.N_
        
        # Iterate through all edges
        for (i, j), w_ij in self.mi.get_all_edges():
            if self.assignments_[i] != self.assignments_[j]:
                self.weight_ += w_ij
                self.diff_weights_[i] += w_ij
                self.diff_weights_[j] += w_ij

    def _update_cut_values(self, index: int, assignments: Optional[List[int]] = None,
                        diff_weights: Optional[List[float]] = None,
                        weight: Optional[float] = None):
        # Use class members if no temporary variables provided
        if assignments is None:
            assignments = self.assignments_
            diff_weights = self.diff_weights_
            weight = self.weight_
            
        # Flip the assignment
        assignments[index] *= -1
        
        # Update objective and difference weights
        for (i, j), w_ij in self.mi.get_all_edges():
            if i == index or j == index:
                other = j if i == index else i
                if assignments[index] != assignments[other]:
                    weight += w_ij
                    diff_weights[index] += w_ij
                    diff_weights[other] += w_ij
                else:
                    weight -= w_ij
                    diff_weights[index] -= w_ij
                    diff_weights[other] -= w_ij

    def _improving_move(self, value) -> bool:
        # For 1-swap moves, value is the index
        if isinstance(value, int):
            return self.diff_weights_[value] > 0
        # For 2-swap moves, value is the benefit
        return value > 0

class Burer2002:
    def __init__(self, mi: MaxCutInstance, runtime_limit: float, 
                 validation: bool, callback=None):
        self.mi = mi
        self.runtime_limit = runtime_limit
        self.validation = validation
        self.callback = callback
        
        # Parameters
        N = 10
        LOCAL_SEARCH = 1
        ONE_MOVE_TOLERANCE = 0.01
        TWO_MOVE_TOLERANCE = 0.1
        PERTURBATION = 0.2

        # Compute 1-norm of weight matrix
        w1norm = sum(2.0 * abs(w) for _, w in mi.get_all_edges())

        iter_count = 0
        while True:
            # Generate random starting angles
            theta = [random.random() * 2 * math.pi for _ in range(mi.get_size())]

            best_weight = float('-inf')
            k = 0  # runs without improvement counter

            while k <= N:
                x = Burer2002Solution.rank2cut(mi, w1norm, theta, self)

                if LOCAL_SEARCH:
                    x.all_1_swap(ONE_MOVE_TOLERANCE)
                    x.all_2_swap(TWO_MOVE_TOLERANCE)

                if not self._report(x, iter_count):
                    return

                if x._improves_over(best_weight):
                    best_weight = x.get_weight()
                    k = 0
                else:
                    k += 1

                # Perturb angles
                for ct in range(mi.get_size()):
                    theta[ct] = (math.pi / 2.0 * (1.0 - x.get_assignments()[ct]) +
                               PERTURBATION * (2 * math.pi * random.random() - math.pi))

            iter_count += 1


class Burer2002Callback:
    def __init__(self):
        self.last_new_best = 0.0
    
    def report(self, solution, new_best: bool, runtime: float, iter: Optional[int] = None) -> bool:
        if new_best:
            print(f"New best {solution.get_weight()} at {runtime}")
            self.last_new_best = runtime
            
        if runtime - self.last_new_best >= 5.0:
            print(f"Exiting at runtime {runtime}")
            return False
        return True
def _report(self, solution: Burer2002Solution, iteration: int) -> bool:
    if self.callback:
        runtime = self._get_runtime()  # You'll need to implement this
        new_best = self._is_new_best(solution)  # You'll need to implement this
        return self.callback.report(solution, new_best, runtime, iteration)
    return True

In [8]:
import networkx as nx
from typing import Dict, Tuple
import time

class NetworkXMaxCutInstance(MaxCutInstance):
    def __init__(self, G: nx.Graph):
        super().__init__()
        self.size = G.number_of_nodes()
        self.edges = {(i, j): float(w.get('weight', 1.0)) 
                     for i, j, w in G.edges(data=True)}

def solve_maxcut(G: nx.Graph, runtime_limit: float = 10.0) -> Dict[int, int]:
    # Create MaxCut instance from NetworkX graph
    instance = NetworkXMaxCutInstance(G)
    
    # Setup callback for runtime tracking
    class TimedCallback(Burer2002Callback):
        def __init__(self):
            super().__init__()
            self.start_time = time.time()
            self.best_solution = None
            
        def report(self, solution, new_best: bool, runtime: float, 
                  iter: Optional[int] = None) -> bool:
            if new_best:
                self.best_solution = solution.get_assignments().copy()
            return super().report(solution, new_best, 
                                time.time() - self.start_time, iter)
    
    callback = TimedCallback()
    
    # Run the algorithm
    solver = Burer2002(instance, runtime_limit, False, callback)
    
    # Convert solution to node dictionary
    return {i: 1 if x == 1 else 0 
            for i, x in enumerate(callback.best_solution)}

# Example usage
if __name__ == "__main__":
    # Create a simple graph
    G = nx.Graph()
    G.add_weighted_edges_from([
        (0, 1, 1.0),
        (1, 2, 2.0),
        (2, 0, -1.0)
    ])
    
    # Solve MaxCut
    partition = solve_maxcut(G, runtime_limit=5.0)
    print(f"Node partition: {partition}")
    
    # Visualize result
    pos = nx.spring_layout(G)
    colors = ['red' if partition[node] == 1 else 'blue' 
              for node in G.nodes()]
    nx.draw(G, pos, node_color=colors, with_labels=True)

AttributeError: 'Burer2002Solution' object has no attribute '_improves_over'